In [ ]:
# | default_exp _code_generator.app_skeleton_generator

In [ ]:
# | export

from typing import *
import time
import json
from pathlib import Path

from yaspin import yaspin

from faststream_gen._components.logger import get_logger
from faststream_gen._code_generator.helper import CustomAIChat, ValidateAndFixResponse, write_file_contents, read_file_contents, validate_python_code
from faststream_gen._code_generator.prompts import APP_SKELETON_GENERATION_PROMPT
from faststream_gen._code_generator.constants import DESCRIPTION_FILE_NAME, \
                                                    APPLICATION_SKELETON_FILE_NAME, \
                                                    GENERATE_APP_SKELETON

In [ ]:

from tempfile import TemporaryDirectory

from faststream_gen._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


def generate_app(
    code_gen_directory: str,
    total_usage: List[Dict[str, int]],
    relevant_prompt_examples: str,
) -> List[Dict[str, int]]:
    """Generate skeleton code for the new FastStream app from the application description

    Args:
        code_gen_directory: The directory containing the generated files.
        total_usage: list of token usage.
        relevant_prompt_examples: Relevant examples to add in the prompts.

    Returns:
        The total token used to generate the FastStream code
    """

    with yaspin(
        text=f"Generating FastStream app skeleton code",
        color="cyan",
        spinner="clock",
    ) as sp:
        app_description_file_name = f"{code_gen_directory}/{DESCRIPTION_FILE_NAME}"
        app_description_content = read_file_contents(app_description_file_name)

        prompt = APP_SKELETON_GENERATION_PROMPT.replace(
            "==== RELEVANT EXAMPLES GOES HERE ====", f"\n{relevant_prompt_examples}"
        )
        app_generator = CustomAIChat(
            params={
                "temperature": 0.5,
            },
            user_prompt=prompt,
            semantic_search_query=app_description_content,
        )
        app_validator = ValidateAndFixResponse(app_generator, validate_python_code)
        validated_app, total_usage = app_validator.fix(
            app_description_content, total_usage
        )

        output_file = f"{code_gen_directory}/{APPLICATION_SKELETON_FILE_NAME}"
        write_file_contents(output_file, validated_app)

        sp.text = ""
        sp.ok(f" ✔ FastStream app skeleton code generated and saved at: {output_file}")
        return total_usage

In [ ]:
# | notest

fixture_description = """
Create a FastStream application using localhost broker for testing using default port number

It should consume from 'store_product' topic an JSON encoded object with the following three attributes: product_name, currency and price. 
The format of the currency will be three letter string, e.g. 'EUR'. For each consumed message, check if the currency attribute is set to 'HRK'. 
If it is then change the currency to 'EUR' and divide the price by 7.5, if the currency is not set to 'HRK' don't change the original message. 
Finally, publish the consumed message to 'change_currency' topic.
"""

relevant_prompt_examples = """
==== EXAMPLE APP DESCRIPTION ====
Create a FastStream application using localhost broker for testing and use the default port number. It should consume messages from the "input_data" topic, where each message is a JSON encoded object containing a single attribute: 'data'. For each consumed message, create a new message object and increment the value of the data attribute by 1. Finally, send the modified message to the 'output_data' topic.

==== YOUR RESPONSE ====
from pydantic import BaseModel, Field, NonNegativeFloat

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class DataBasic(BaseModel):
    data: NonNegativeFloat = Field(
        ..., examples=[0.5], description="Float data example"
    )


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.publisher("output_data")
@broker.subscriber("input_data")
async def on_input_data(msg: DataBasic, logger: Logger) -> DataBasic:
    '''Processes a message from 'input_data' topic, increments 'data' attribute by 1, and sends it to 'output_data'.

    Instructions:
    1. Consume a message from 'input_data' topic.
    2. Log the consumed message using logger.info
    3. Create a new message object (do not directly modify the original).
    4. Increment 'data' attribute by 1 in the new message.
    5. Send the modified message to 'output_data' topic.


    '''
    raise NotImplementedError()
"""
                
with TemporaryDirectory() as d:
    output_path = f"{str(d)}/fastkafka-gen"
    output_file = f"{output_path}/{APPLICATION_SKELETON_FILE_NAME}"
    description_file = f"{output_path}/{DESCRIPTION_FILE_NAME}"    
    write_file_contents(description_file, fixture_description)
    
    usage = generate_app(output_path, [], relevant_prompt_examples)
    
    assert Path(output_path).exists()
    
    actual = [file for file in Path(output_path).iterdir()]
    print(actual)
    assert len(actual) == 2
    
    contents = read_file_contents(output_file)
    print(contents)

assert int(usage[0]["total_tokens"]) > 0
print(usage)

⠋ Generating FastStream app skeleton code[INFO] faiss.loader: Loading faiss with AVX2 support.
[INFO] faiss.loader: Successfully loaded faiss with AVX2 support.
⠙ Generating FastStream app skeleton code 

/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:119: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


 ✔ FastStream app skeleton code generated and saved at: /tmp/tmprl7dm137/fastkafka-gen/application_skeleton.py 
[PosixPath('/tmp/tmprl7dm137/fastkafka-gen/app_description.txt'), PosixPath('/tmp/tmprl7dm137/fastkafka-gen/application_skeleton.py')]
from pydantic import BaseModel, Field

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class Product(BaseModel):
    product_name: str = Field(..., description="Name of the product")
    currency: str = Field(..., description="Currency of the price")
    price: float = Field(..., description="Price of the product")


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.publisher("change_currency")
@broker.subscriber("store_product")
async def on_store_product(product: Product, logger: Logger) -> Product:
    '''Processes a message from 'store_product' topic, changes currency to 'EUR' and divides price by 7.5 if currency is 'HRK'.

    Instructions:
    1. Consume a message from 'store_p

In [ ]:
# # | notest

# fixture_app_skeleton = """
# from pydantic import BaseModel, Field

# from faststream import FastStream, Logger
# from faststream.kafka import KafkaBroker


# class Product(BaseModel):
#     product_name: str = Field(
#         ..., examples=["Apple"], description="Product name example"
#     )
#     currency: str = Field(
#         ..., examples=["HRK"], description="Currency example"
#     )
#     price: float = Field(
#         ..., examples=[10.0], description="Price example"
#     )


# broker = KafkaBroker("localhost:9092")
# app = FastStream(broker)


# @broker.publisher("change_currency")
# @broker.subscriber("store_product")
# async def on_store_product(msg: Product, logger: Logger) -> Product:
#     '''Process messages from the 'store_product' topic and publish to 'change_currency' topic.

#     Args:
#         input_data (dict): A JSON-encoded message with 'product_name', 'currency', and 'price' attributes.

#     Instructions:
#     1. Create a new instance of the message object from the 'input_data'.
#     2. Log the consumed message using logger.info
#     3. Do not directly modify the input message object.
#     4. Check if the 'currency' attribute is set to 'HRK' in the input message.
#     5. If 'currency' is set to 'HRK':
#         - Change the 'currency' attribute to 'EUR'.
#         - Divide the 'price' attribute by 7.5.
#     6. Publish the modified message to the 'change_currency' topic.

#     '''
#     raise NotImplementedError()
# """

# relevant_prompt_examples = """
# ==== EXAMPLE APP SKELETON ====

# from pydantic import BaseModel, Field, NonNegativeFloat

# from faststream import FastStream, Logger
# from faststream.kafka import KafkaBroker


# class DataBasic(BaseModel):
#     data: NonNegativeFloat = Field(
#         ..., examples=[0.5], description="Float data example"
#     )


# broker = KafkaBroker("localhost:9092")
# app = FastStream(broker)


# @broker.publisher("output_data")
# @broker.subscriber("input_data")
# async def on_input_data(msg: DataBasic, logger: Logger) -> DataBasic:
#     '''Processes a message from 'input_data' topic, increments 'data' attribute by 1, and sends it to 'output_data'.

#     Instructions:
#     1. Consume a message from 'input_data' topic.
#     2. Log the consumed message using logger.info
#     3. Create a new message object (do not directly modify the original).
#     4. Increment 'data' attribute by 1 in the new message.
#     5. Send the modified message to 'output_data' topic.

#     '''
#     raise NotImplementedError()


# ==== YOUR RESPONSE ====

# from pydantic import BaseModel, Field, NonNegativeFloat

# from faststream import FastStream, Logger
# from faststream.kafka import KafkaBroker


# class DataBasic(BaseModel):
#     data: NonNegativeFloat = Field(
#         ..., examples=[0.5], description="Float data example"
#     )


# broker = KafkaBroker("localhost:9092")
# app = FastStream(broker)


# @broker.publisher("output_data")
# @broker.subscriber("input_data")
# async def on_input_data(msg: DataBasic, logger: Logger) -> DataBasic:
#     logger.info(msg)
#     return DataBasic(data=msg.data + 1.0)
# """
                
# with TemporaryDirectory() as d:
#     output_path = f"{str(d)}/fastkafka-gen"
#     output_file = f"{output_path}/{APPLICATION_FILE_NAME}"
#     app_skeleton_file = f"{output_path}/{APPLICATION_SKELETON_FILE_NAME}"    
#     write_file_contents(app_skeleton_file, fixture_app_skeleton)
    
#     usage = generate_app(output_path, [], relevant_prompt_examples, GENERATE_APP_FROM_SKELETON)
    
#     assert Path(output_path).exists()
    
#     actual = [file for file in Path(output_path).iterdir()]
#     print(actual)
#     assert len(actual) == 2
    
#     contents = read_file_contents(output_file)
#     print(contents)

# assert int(usage[0]["total_tokens"]) > 0
# print(usage)

In [ ]:
# # | notest

# fixture_spec = '''
# asyncapi: 2.5.0
# info:
#   title: Product currency converter
#   version: 0.0.1
#   description: 'A FastKafka application using localhost broker for testing, staging.airt.ai
#     for staging and prod.airt.ai for production, using default port numbers. It should
#     consume from ''store_product'' topic an JSON encoded object with the following
#     three attributes: product_name, currency and price. The format of the currency
#     will be three letter string, e.g. ''EUR''. For each consumed message, check if
#     the currency attribute is set to ''HRK''. If it is then change the currency to
#     ''EUR'' and divide the price by 7.5, if the currency is not set to ''HRK'' don''t
#     change the original message. Finally, publish the consumed message to ''change_currency''
#     topic. Use SASL_SSL with SCRAM-SHA-256 for authentication.'
#   contact:
#     name: Author
#     url: https://www.google.com/
#     email: noreply@gmail.com
# servers:
#   localhost:
#     url: localhost
#     description: local development kafka broker
#     protocol: kafka
#     variables:
#       port:
#         default: '9092'
#   staging:
#     url: staging.airt.ai
#     description: staging kafka broker
#     protocol: kafka-secure
#     security:
#     - staging_default_security: []
#     variables:
#       port:
#         default: '9092'
#   production:
#     url: prod.airt.ai
#     description: production kafka broker
#     protocol: kafka-secure
#     security:
#     - production_default_security: []
#     variables:
#       port:
#         default: '9092'
# channels:
#   store_product:
#     subscribe:
#       message:
#         $ref: '#/components/messages/StoreProduct'
#       description: For each consumed message, check if the currency attribute is set
#         to 'HRK'. If it is then change the currency to 'EUR', then make a new instance of the message and only change the price attribute
#         by dividing it by 7.5, if the currency is not set to 'HRK' don't change the original message.
#         Finally, publish the consumed message to 'change_currency' topic.
#   change_currency:
#     publish:
#       message:
#         $ref: '#/components/messages/StoreProduct'
# components:
#   messages:
#     StoreProduct:
#       payload:
#         properties:
#           product_name:
#             description: Name of the product.
#             title: Product Name
#             type: string
#           currency:
#             description: The currency.
#             title: Currency
#             type: string
#           price:
#             description: Price of the product.
#             title: Price
#             type: number
#         required:
#         - product_name
#         - currency
#         - price
#         title: StoreProduct
#         type: object
#   schemas: {}
#   securitySchemes:
#     staging_default_security:
#       type: scramSha256
#     production_default_security:
#       type: scramSha256

# '''

# with TemporaryDirectory() as d:
#     output_path = f"{str(d)}/fastkafka-gen"
#     output_file = f"{output_path}/{APPLICATION_FILE_NAME}"
#     spec_file = f"{output_path}/{ASYNC_API_SPEC_FILE_NAME}"    
#     write_file_contents(spec_file, fixture_spec)
    
#     usage = generate_app(output_path, [])
    
#     assert Path(output_path).exists()
    
#     actual = [file for file in Path(output_path).iterdir()]
#     print(actual)
#     assert len(actual) == 2
    
#     contents = read_file_contents(output_file)
#     print(contents)

# assert int(usage[0]["total_tokens"]) > 0
# print(usage)